In [1]:
from transformers import GPTNeoXForCausalLM, PreTrainedTokenizerFast
import torch

model_ckpt = 'beomi/KoAlpaca-Polyglot'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
model = GPTNeoXForCausalLM.from_pretrained(model_ckpt).to(device)

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_ckpt)

In [4]:
prompt = """\
    |로 구분된 키워드와 관련된 채용 공고를 생성해주세요.
    키워드를 |로 구분했을 때 첫번째 키워드는 전반적인 채용 공고의 방향성을 나타냅니다.
    키워드를 |로 구분했을 때 첫번째 키워드를 제외한 나머지 키워드는 채용공고에 들어가야 하는 필수 키워드입니다.
    
    예시)
    키워드: 업무 내용|파이썬|인공지능|자연어처리
    채용공고: -python을 활용한 인공지능 알고리즘 개발\n -python을 활용한 장비 통신\n -데이터분석,인공지능 알고리즘 개발 등\n -Python, 딥러닝 관련 개발
    
    키워드: 업무내용|트랜스포머|AI|NLP
    """

In [5]:
prompt = """\
    키워드와 관련된 채용 공고를 생성해주세요.
    
    키워드: 업무 내용|파이썬|인공지능|자연어처리
    채용공고: -python을 활용한 인공지능 알고리즘 개발
    -python을 활용한 장비 통신
    -데이터분석,인공지능 알고리즘 개발 등
    -Python, 딥러닝 관련 개발
    
    키워드: 업무내용|트랜스포머|AI|NLP
    채용공고:
    """

In [6]:
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

In [7]:
decode_params = {
    "input_ids": input_ids,
    "max_length": 128,
    "num_beams": 2, #용량 문제
    "do_sample": False,
    "no_repeat_ngram_size": 2,
    "attention_mask": attention_mask
}

In [ ]:
output_beam = model.generate(input_ids, attention_mask=attention_mask, max_length=decode_params["max_length"], num_beams=decode_params['num_beams'], do_sample=decode_params['do_sample'], no_repeat_ngram_size=decode_params['no_repeat_ngram_size'])

In [ ]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

In [ ]:
def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:,1:]
        )
        
        seq_log_prob = torch.sum(log_probs[:, input_len:])
        
    return seq_log_prob.cpu().numpy()

In [ ]:
logp=sequence_logprob(model, output_beam, input_len=len(input_ids[0]))

In [ ]:
print(tokenizer.decode(output_beam[0]))

In [ ]:
print(f"\n로그확률: {logp:.2f}")